# Code for Project Management

### Example:

\begin{array}{rr} \hline
\text{Activity} &\text{Successors} &\text{Duration} &\text{Crash Cost} & \text{Crash Capacity}  \\ \hline
1 & 2    & 3 & 1 & 3  \\ 
2 & 3,4    & 1 & 0 & 0  \\ 
3 & 5    & 1 & 0 & 0 \\ 
4 & 6    & 1 & 2 & 1/2 \\ 
5 & 6,7    & 1 & 2 & 1/2\\ 
6 & 8 & 2 & 2 & 1\\ 
7 & 9    & 2 & 2 & 1/2  \\ 
8 & 9    & 1 & 0 & 0 \\ 
9 & - & 1 & 2 & 1/2 \\ 
\hline
\end{array}

In [9]:
from gurobipy import *
import numpy as np

#########Parameters Set-up############

NODES = np.arange(1, 11)

N_NODES = len(NODES)

duration = np.array([3,1,1,1,1,2,2,1,1,0])

cost = np.array([1,0,0,2,2,2,2,0,2,0])

capacity = np.array([3,0,0,0.5,0.5,1,0.5,0,0.5,0])

NODES, duration, cost, capacity = multidict( {NODES[i]: (duration[i], cost[i], capacity[i]) for i in range(N_NODES) })

ARCS = tuplelist([(1,2), (2,3), (2,4), (3,5), (4,6), (5,6), (5,7), (6,8), (7,9), (8,9), (9,10)])

print("\n Nodes set is:", NODES)
print("\n The duration is:", duration)
print("\n The capacity is:", capacity)                                            
print("\n The arc set is: \n", ARCS)

N_ARCS = len(ARCS)

deadline = 7


 Nodes set is: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

 The duration is: {1: 3, 2: 1, 3: 1, 4: 1, 5: 1, 6: 2, 7: 2, 8: 1, 9: 1, 10: 0}

 The capacity is: {1: 3.0, 2: 0.0, 3: 0.0, 4: 0.5, 5: 0.5, 6: 1.0, 7: 0.5, 8: 0.0, 9: 0.5, 10: 0.0}

 The arc set is: 
 <gurobi.tuplelist (11 tuples, 2 values each):
 ( 1 , 2  )
 ( 2 , 3  )
 ( 2 , 4  )
 ( 3 , 5  )
 ( 4 , 6  )
 ( 5 , 6  )
 ( 5 , 7  )
 ( 6 , 8  )
 ( 7 , 9  )
 ( 8 , 9  )
 ( 9 , 10 )
>


In [10]:
#########Model Set-up###############

pm = Model("project_management")

# Creat variables
# the crashing decisions
x = pm.addVars(NODES, lb=0.0, ub=capacity)
# The starting time of each project
y = pm.addVars(NODES)

# Set objective
pm.setObjective( quicksum(cost[i]*x[i] for i in NODES), GRB.MINIMIZE)

# Add precedence constraints: 
pm.addConstrs(( y[i] - (y[j] + duration[j] - x[j]) >= 0 for (j, i) in ARCS ), "Precedence")

# Add deadline constraint: 
pm.addConstr(y[N_NODES] <= deadline, "Deadline")

# Solving the model
pm.optimize()

#  Print optimal solutions and optimal value
print("\n Crashing decisions:")
for i in NODES:
    print("\n The duration of activity %g is crashed by %g weeks;" % (i, x[i].x))
    
print("\n Project duration:")
for i in NODES:
    print("The starting of activity %g is %g th week;"  % (i, y[i].x))
    
print('Obj:', pm.objVal)


Optimize a model with 12 rows, 20 columns and 34 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [5e-01, 3e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 6 rows and 11 columns
Presolve time: 0.16s
Presolved: 6 rows, 9 columns, 17 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.507000e+00   0.000000e+00      0s
       4    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.23 seconds
Optimal objective  3.000000000e+00

 Crashing decisions:

 The duration of activity 1 is crashed by 3 weeks;

 The duration of activity 2 is crashed by 0 weeks;

 The duration of activity 3 is crashed by 0 weeks;

 The duration of activity 4 is crashed by 0 weeks;

 The duration of activity 5 is crashed by 0 weeks;

 The duration of activity 6 is crashed by 0 weeks;

 The duration of activity 7 is crashed by 0 weeks;

 The duration of activit